In [1]:
import argparse
import datetime
import glob
import hashlib
import json
import multiprocessing
import pickle
import os
import shutil
import subprocess
import uuid
import random

import numpy as np
import pandas as pd
import lightgbm as lgb
from collections import defaultdict
from lightgbm.sklearn import LGBMRanker
from tqdm import tqdm

from pyserini.analysis import Analyzer, get_lucene_analyzer
from pyserini.ltr import *
from pyserini.search import get_topics_with_reader

def train_data_loader(task='triple', neg_sample=10, random_seed=12345):
    if os.path.exists(f'train_{task}_sampled_with_{neg_sample}_{random_seed}.pickle'):
        sampled_train = pd.read_pickle(f'train_{task}_sampled_with_{neg_sample}_{random_seed}.pickle')
        print(sampled_train.shape)
        print(sampled_train.index.get_level_values('qid').drop_duplicates().shape)
        print(sampled_train.groupby('qid').count().mean())
        print(sampled_train.head(10))
        print(sampled_train.info())
        return sampled_train
    else:
        if task == 'triple':
            train = pd.read_csv('collections/msmarco-passage/qidpidtriples.train.full.2.tsv', sep="\t",
                                names=['qid', 'pos_pid', 'neg_pid'], dtype=np.int32)
            pos_half = train[['qid', 'pos_pid']].rename(columns={"pos_pid": "pid"}).drop_duplicates()
            pos_half['rel'] = np.int32(1)
            neg_half = train[['qid', 'neg_pid']].rename(columns={"neg_pid": "pid"}).drop_duplicates()
            neg_half['rel'] = np.int32(0)
            del train
            sampled_neg_half = []
            for qid, group in tqdm(neg_half.groupby('qid')):
                sampled_neg_half.append(group.sample(n=min(neg_sample, len(group)), random_state=random_seed))
            sampled_train = pd.concat([pos_half] + sampled_neg_half, axis=0, ignore_index=True)
            sampled_train = sampled_train.sort_values(['qid','pid']).set_index(['qid','pid'])
            print(sampled_train.shape)
            print(sampled_train.index.get_level_values('qid').drop_duplicates().shape)
            print(sampled_train.groupby('qid').count().mean())
            print(sampled_train.head(10))
            print(sampled_train.info())

            sampled_train.to_pickle(f'train_{task}_sampled_with_{neg_sample}_{random_seed}.pickle')
        elif task == 'rank':
            qrel = defaultdict(list)
            with open("collections/msmarco-passage/qrels.train.tsv") as f:
                for line in f:
                    topicid, _, docid, rel = line.strip().split('\t')
                    assert rel == "1", line.split(' ')
                    qrel[topicid].append(docid)
            
            qid2pos = defaultdict(list)
            qid2neg = defaultdict(list)
            with open("runs/msmarco-passage/run.train.small.tsv") as f:
                for line in tqdm(f):
                    topicid, docid, rank = line.split()
                    assert topicid in qrel
                    if docid in qrel[topicid]:
                        qid2pos[topicid].append(docid)
                    else:
                        qid2neg[topicid].append(docid)
            sampled_train = []
            for topicid, pos_list in tqdm(qid2pos.items()):
                neg_list = random.sample(qid2neg[topicid], min(len(qid2neg[topicid]), neg_sample))
                for positive_docid in pos_list:
                    sampled_train.append((int(topicid), int(positive_docid), 1))
                for negative_docid in neg_list:
                    sampled_train.append((int(topicid), int(negative_docid), 0))
            sampled_train = pd.DataFrame(sampled_train,columns=['qid','pid','rel'],dtype=np.int32)
            sampled_train = sampled_train.sort_values(['qid','pid']).set_index(['qid','pid'])
            print(sampled_train.shape)
            print(sampled_train.index.get_level_values('qid').drop_duplicates().shape)
            print(sampled_train.groupby('qid').count().mean())
            print(sampled_train.head(10))
            print(sampled_train.info())

            sampled_train.to_pickle(f'train_{task}_sampled_with_{neg_sample}_{random_seed}.pickle')
        else:
            raise Exception('unknown parameters')
        return sampled_train
sampled_train = train_data_loader(task='triple', neg_sample=10)

(39738110, 1)
(400782,)
rel    99.151434
dtype: float64
            rel
qid pid        
3   6190      0
    297633    0
    464784    0
    489427    0
    521826    0
    648464    0
    689698    0
    771941    0
    811866    0
    970814    0
<class 'pandas.core.frame.DataFrame'>
MultiIndex: 39738110 entries, (3, 6190) to (1185869, 8626607)
Data columns (total 1 columns):
 #   Column  Dtype
---  ------  -----
 0   rel     int32
dtypes: int32(1)
memory usage: 848.2 MB
None


In [2]:
def dev_data_loader(task='pygaggle'):
    if os.path.exists(f'dev_{task}.pickle'):
        dev = pd.read_pickle(f'dev_{task}.pickle')
        print(dev.shape)
        print(dev.index.get_level_values('qid').drop_duplicates().shape)
        print(dev.groupby('qid').count().mean())
        print(dev.head(10))
        print(dev.info())
        dev_qrel = pd.read_pickle(f'dev_qrel.pickle')
        return dev, dev_qrel
    else:
        if task == 'rerank':
            dev = pd.read_csv('collections/msmarco-passage/top1000.dev', sep="\t",
                              names=['qid', 'pid', 'query', 'doc'], usecols=['qid', 'pid'], dtype=np.int32)
        elif task == 'anserini':
            dev = pd.read_csv('runs/msmarco-passage/run.msmarco-passage.dev.small.tsv',sep="\t",
                            names=['qid','pid','rank'], dtype=np.int32)
        elif task == 'pygaggle':
            dev = pd.read_csv('../pygaggle/data/msmarco_ans_entire/run.dev.small.tsv',sep="\t",
                            names=['qid','pid','rank'], dtype=np.int32)
        else:
            raise Exception('unknown parameters')
        dev_qrel = pd.read_csv('collections/msmarco-passage/qrels.dev.small.tsv', sep="\t",
                               names=["qid", "q0", "pid", "rel"], usecols=['qid', 'pid', 'rel'], dtype=np.int32)
        dev = dev.merge(dev_qrel, left_on=['qid', 'pid'], right_on=['qid', 'pid'], how='left')
        dev['rel'] = dev['rel'].fillna(0).astype(np.int32)
        dev = dev.sort_values(['qid','pid']).set_index(['qid','pid'])
        
        print(dev.shape)
        print(dev.index.get_level_values('qid').drop_duplicates().shape)
        print(dev.groupby('qid').count().mean())
        print(dev.head(10))
        print(dev.info())

        dev.to_pickle(f'dev_{task}.pickle')
        dev_qrel.to_pickle(f'dev_qrel.pickle')
        return dev, dev_qrel
dev, dev_qrel = dev_data_loader(task='pygaggle')

(6974598, 2)
(6980,)
rank    999.226074
rel     999.226074
dtype: float64
            rank  rel
qid pid              
2   55860    345    0
    72202    557    0
    72210    213    0
    98589    278    0
    98590    323    0
    98593    580    0
    98595    553    0
    112123   108    0
    112126   469    0
    112127    21    0
<class 'pandas.core.frame.DataFrame'>
MultiIndex: 6974598 entries, (2, 55860) to (1102400, 8830447)
Data columns (total 2 columns):
 #   Column  Dtype
---  ------  -----
 0   rank    int32
 1   rel     int32
dtypes: int32(2)
memory usage: 282.7 MB
None


In [3]:
def query_loader(choice='default'):
    if os.path.exists(f'query_{choice}_tokenized.pickle'):
        return pickle.load(open(f'query_{choice}_tokenized.pickle','rb'))
    else:
        if choice == 'default':
            analyzer = Analyzer(get_lucene_analyzer())
            nonStopAnalyzer = Analyzer(get_lucene_analyzer(stopwords=False))
            queries = get_topics_with_reader('io.anserini.search.topicreader.TsvIntTopicReader', \
                                             'collections/msmarco-passage/queries.train.tsv')
            queries.update(get_topics_with_reader('io.anserini.search.topicreader.TsvIntTopicReader', \
                                                  'collections/msmarco-passage/queries.dev.tsv'))
            for qid,value in queries.items():
                assert 'tokenized' not in value
                value['tokenized'] = analyzer.analyze(value['title'])
                assert 'nonSW' not in value
                value['nonSW'] = nonStopAnalyzer.analyze(value['title'])
        else:
            raise Exception('unknown parameters')

        pickle.dump(queries,open(f'query_{choice}_tokenized.pickle','wb'))

        return queries
queries = query_loader()

In [4]:
fe = FeatureExtractor('../indexes/msmarco-passage/lucene-index-msmarco/',max(multiprocessing.cpu_count()//2,1))
fe.add(BM25(k1=0.9,b=0.4))
fe.add(BM25(k1=1.2,b=0.75))
fe.add(BM25(k1=2.0,b=0.75))

fe.add(LMDir(mu=1000))
fe.add(LMDir(mu=1500))
fe.add(LMDir(mu=2500))

fe.add(LMJM(0.1))
fe.add(LMJM(0.4))
fe.add(LMJM(0.7))

fe.add(NTFIDF())
fe.add(ProbalitySum())

fe.add(DFR_GL2())
fe.add(DFR_In_expB2())
fe.add(DPH())

fe.add(ContextDFR_GL2(AvgPooler()))
fe.add(ContextDFR_GL2(MinPooler()))
fe.add(ContextDFR_GL2(MaxPooler()))
fe.add(ContextDFR_In_expB2(AvgPooler()))
fe.add(ContextDFR_In_expB2(MinPooler()))
fe.add(ContextDFR_In_expB2(MaxPooler()))
fe.add(ContextDPH(AvgPooler()))
fe.add(ContextDPH(MinPooler()))
fe.add(ContextDPH(MaxPooler()))

fe.add(Proximity())
fe.add(TPscore())
fe.add(tpDist())

fe.add(DocSize())
fe.add(Entropy())
fe.add(StopCover())
fe.add(StopRatio())

fe.add(QueryLength())
fe.add(QueryLengthNonStopWords())
fe.add(QueryCoverageRatio())
fe.add(UniqueTermCount())
fe.add(MatchingTermCount())
fe.add(SCS())

fe.add(tfStat(AvgPooler()))
fe.add(tfStat(SumPooler()))
fe.add(tfStat(MinPooler()))
fe.add(tfStat(MaxPooler()))
fe.add(tfStat(VarPooler()))
fe.add(tfIdfStat(AvgPooler()))
fe.add(tfIdfStat(SumPooler()))
fe.add(tfIdfStat(MinPooler()))
fe.add(tfIdfStat(MaxPooler()))
fe.add(tfIdfStat(VarPooler()))
fe.add(scqStat(AvgPooler()))
fe.add(scqStat(SumPooler()))
fe.add(scqStat(MinPooler()))
fe.add(scqStat(MaxPooler()))
fe.add(scqStat(VarPooler()))
fe.add(normalizedTfStat(AvgPooler()))
fe.add(normalizedTfStat(SumPooler()))
fe.add(normalizedTfStat(MinPooler()))
fe.add(normalizedTfStat(MaxPooler()))
fe.add(normalizedTfStat(VarPooler()))

fe.add(idfStat(AvgPooler()))
fe.add(idfStat(SumPooler()))
fe.add(idfStat(MinPooler()))
fe.add(idfStat(MaxPooler()))
fe.add(idfStat(VarPooler()))
fe.add(idfStat(MaxMinRatioPooler()))
fe.add(idfStat(ConfidencePooler()))
fe.add(ictfStat(AvgPooler()))
fe.add(ictfStat(SumPooler()))
fe.add(ictfStat(MinPooler()))
fe.add(ictfStat(MaxPooler()))
fe.add(ictfStat(VarPooler()))
fe.add(ictfStat(MaxMinRatioPooler()))
fe.add(ictfStat(ConfidencePooler()))

fe.add(UnorderedSequentialPairs(3))
fe.add(UnorderedSequentialPairs(8))
fe.add(UnorderedSequentialPairs(15))
fe.add(OrderedSequentialPairs(3))
fe.add(OrderedSequentialPairs(8))
fe.add(OrderedSequentialPairs(15))
fe.add(UnorderedQueryPairs(3))
fe.add(UnorderedQueryPairs(8))
fe.add(UnorderedQueryPairs(15))
fe.add(OrderedQueryPairs(3))
fe.add(OrderedQueryPairs(8))
fe.add(OrderedQueryPairs(15))

In [5]:
def extract(df, queries, fe):
    df_pieces = []
    fetch_later = []
    qidpid2rel = defaultdict(dict)
    need_rows = 0
    for qid,group in tqdm(df.groupby('qid')):
        for t in group.reset_index().itertuples():
            assert t.pid not in qidpid2rel[t.qid]
            qidpid2rel[t.qid][t.pid] = t.rel
            need_rows += 1
        #test.py has bug here, it does not convert pid to str, not sure why it does not cause problem in java
        fe.lazy_extract(str(qid),
                        queries[qid]['nonSW'], 
                        queries[qid]['tokenized'],
                        [str(pid) for pid in qidpid2rel[t.qid].keys()])
        fetch_later.append(str(qid))
        if len(fetch_later) == 10000:
            info = np.zeros(shape=(need_rows,3), dtype=np.int32)
            feature = np.zeros(shape=(need_rows,len(fe.feature_names())), dtype=np.float32)
            idx = 0
            for qid in fetch_later:
                for doc in fe.get_result(qid):
                    info[idx,0] = int(qid)
                    info[idx,1] = int(doc['pid'])
                    info[idx,2] = qidpid2rel[int(qid)][int(doc['pid'])]
                    feature[idx,:] = doc['features']
                    idx += 1
            info = pd.DataFrame(info, columns=['qid','pid','rel'])
            feature = pd.DataFrame(feature, columns=fe.feature_names())
            df_pieces.append(pd.concat([info,feature], axis=1))
            fetch_later = []
            need_rows = 0
    #deal with rest
    if len(fetch_later) > 0:
        info = np.zeros(shape=(need_rows,3), dtype=np.int32)
        feature = np.zeros(shape=(need_rows,len(fe.feature_names())), dtype=np.float32)
        idx = 0
        for qid in fetch_later:
            for doc in fe.get_result(qid):
                info[idx,0] = int(qid)
                info[idx,1] = int(doc['pid'])
                info[idx,2] = qidpid2rel[int(qid)][int(doc['pid'])]
                feature[idx,:] = doc['features']
                idx += 1
        info = pd.DataFrame(info, columns=['qid','pid','rel'])
        feature = pd.DataFrame(feature, columns=fe.feature_names())
        df_pieces.append(pd.concat([info,feature], axis=1))
    data = pd.concat(df_pieces, axis=0, ignore_index=True)
    data = data.sort_values(by='qid', kind='mergesort')
    group = data.groupby('qid').agg(count=('pid', 'count'))['count']
    return data,group

In [6]:
def hash_df(df):
    h = pd.util.hash_pandas_object(df)
    return hex(h.sum().astype(np.uint64))


def hash_anserini_jar():
    find = glob.glob(os.environ['ANSERINI_CLASSPATH'] + "/*fatjar.jar")
    assert len(find) == 1
    md5Hash = hashlib.md5(open(find[0], 'rb').read())
    return md5Hash.hexdigest()


def hash_fe(fe):
    return hashlib.md5(','.join(sorted(fe.feature_names())).encode()).hexdigest()


def data_loader(task, df, queries, fe):
    df_hash = hash_df(df)
    jar_hash = hash_anserini_jar()
    fe_hash = hash_fe(fe)
    if os.path.exists(f'{task}_{df_hash}_{jar_hash}_{fe_hash}.pickle'):
        res = pickle.load(open(f'{task}_{df_hash}_{jar_hash}_{fe_hash}.pickle','rb'))
        print(res['data'].shape)
        print(res['data'].qid.drop_duplicates().shape)
        print(res['group'].mean())
        print(res['data'].head(10))
        print(res['data'].info())
        return res
    else:
        if task == 'train' or task == 'dev': 
            data,group = extract(df, queries, fe)
            obj = {'data':data,'group':group,'df_hash':df_hash,'jar_hash':jar_hash,'fe_hash':fe_hash}
            print(data.shape)
            print(data.qid.drop_duplicates().shape)
            print(group.mean())
            print(data.head(10))
            print(data.info())
            pickle.dump(obj,open(f'{task}_{df_hash}_{jar_hash}_{fe_hash}.pickle','wb'))
            return obj
        else:
            raise Exception('unknown parameters')

In [7]:
import json
def export(df, fn):
    with open(fn,'w') as f:
        for qid, group in tqdm(df.groupby('qid')):
            line = {}
            line['qid'] = qid
            line['queryTokens'] = queries[qid]['tokenized']
            line['queryText'] = queries[qid]['nonSW']
            line['docIds'] = [str(did) for did in group.reset_index().pid.drop_duplicates().tolist()]
            f.write(json.dumps(line)+'\n')

In [8]:
train_extracted = data_loader('train', sampled_train, queries, fe)
# export(sampled_train, 'sampled_train_export.json')
dev_extracted = data_loader('dev', dev, queries, fe)
# export(dev, 'sampled_dev_export.json')
del sampled_train, dev

(39738110, 76)
(400782,)
99.15143394663433
   qid     pid  rel  contents_BM25_k1_0.90_b_0.40  \
0    3    6190    0                     17.671618   
1    3  297633    0                     23.997044   
2    3  464784    0                     17.301842   
3    3  489427    0                     19.158892   
4    3  521826    0                     14.119576   
5    3  648464    0                     15.092525   
6    3  689698    0                     16.065151   
7    3  771941    0                     14.379218   
8    3  811866    0                     23.821770   
9    3  970814    0                     17.063637   

   contents_BM25_k1_1.20_b_0.75  contents_BM25_k1_2.00_b_0.75  \
0                     19.488779                     21.927422   
1                     27.062508                     30.874779   
2                     17.945347                     20.363523   
3                     21.008556                     21.883081   
4                     15.704762                 

(6974598, 76)
(6980,)
999.2260744985673
   qid     pid  rel  contents_BM25_k1_0.90_b_0.40  \
0    2   55860    0                     12.348820   
1    2   72202    0                     10.927653   
2    2   72210    0                     13.675473   
3    2   98589    0                     12.699286   
4    2   98590    0                     12.492470   
5    2   98593    0                     11.077914   
6    2   98595    0                     11.181725   
7    2  112123    0                     15.955744   
8    2  112126    0                     11.468307   
9    2  112127    0                     21.200821   

   contents_BM25_k1_1.20_b_0.75  contents_BM25_k1_2.00_b_0.75  \
0                     13.151269                     14.964364   
1                     11.041017                     11.088634   
2                     13.716912                     14.579938   
3                     14.042357                     16.412817   
4                     13.508439                    

In [9]:
def eval_mrr(dev_data):
    score_tie_counter = 0
    score_tie_query = set()

    MRR = []
    for qid, group in tqdm(dev_data.groupby('qid')):
        group = group.reset_index()
        rank = 0
        prev_score = None
        assert len(group['pid'].tolist()) == len(set(group['pid'].tolist()))
        # stable sort is also used in LightGBM

        for t in group.sort_values('score', ascending=False, kind='mergesort').itertuples():
            if prev_score is not None and abs(t.score - prev_score) < 1e-8:
                score_tie_counter += 1
                score_tie_query.add(qid)
            prev_score = t.score
            prev_pid = t.pid
            rank += 1
            if t.rel>0:
                MRR.append(1.0/rank)
                break
            elif rank == 10 or rank == len(group):
                MRR.append(0.)
                break

    score_tie = f'score_tie occurs {score_tie_counter} times in {len(score_tie_query)} queries'
    print(score_tie,np.mean(MRR))


In [10]:
def eval_recall(dev_qrel, dev_data):
    dev_rel_num = dev_qrel[dev_qrel['rel']>0].groupby('qid').count()['rel']

    score_tie_counter = 0
    score_tie_query = set()
    
    recall_point = [10,20,50,100,200,500,1000]
    recall_curve = {k:[] for k in recall_point}
    for qid, group in tqdm(dev_data.groupby('qid')):
        group = group.reset_index()
        rank = 0
        prev_score = None
        assert len(group['pid'].tolist()) == len(set(group['pid'].tolist()))
        # stable sort is also used in LightGBM
        total_rel = dev_rel_num.loc[qid]
        query_recall = [0 for k in recall_point]
        for t in group.sort_values('score', ascending=False, kind='mergesort').itertuples():
            if prev_score is not None and abs(t.score - prev_score) < 1e-8:
                score_tie_counter += 1
                score_tie_query.add(qid)
            prev_score = t.score
            rank += 1
            if t.rel>0:
                for i,p in enumerate(recall_point):
                    if rank <= p:
                        query_recall[i] += 1
        for i,p in enumerate(recall_point):
            if total_rel>0:
                recall_curve[p].append(query_recall[i]/total_rel)
            else:
                recall_curve[p].append(0.)

    score_tie = f'score_tie occurs {score_tie_counter} times in {len(score_tie_query)} queries'
    print(score_tie)
    
    for k,v in recall_curve.items():
        avg = np.mean(v)
        print(f'recall@{k}:{avg}')


In [11]:
feature_name = fe.feature_names()
train_X = train_extracted['data'].loc[:, feature_name]
train_Y = train_extracted['data']['rel']
dev_X = dev_extracted['data'].loc[:, feature_name]
dev_Y = dev_extracted['data']['rel']
lgb_train = lgb.Dataset(train_X,label=train_Y,group=train_extracted['group'])
lgb_valid = lgb.Dataset(dev_X,label=dev_Y,group=dev_extracted['group'])

In [12]:
for i,n in enumerate(feature_name):
    if np.isnan(train_X.iloc[:,i]).any():
        print(n)

contents_DPH
Entropy
contents_IDF_confidence
contents_ICTF_confidence


In [13]:
dev_rel_num = dev_qrel[dev_qrel['rel']>0].groupby('qid').count()['rel']
gid = 0
sample_id = 0
dev_group_rel_num = []
for qid,group in dev_extracted['data'].groupby('qid'):
    group = group.sort_values(['pid'])
    assert len(group) == dev_extracted['group'].iloc[gid]
    assert np.isclose(group.iloc[0,:].loc[feature_name],
                      dev_X.iloc[sample_id,:], equal_nan=True).all()
    dev_group_rel_num.append(dev_rel_num.loc[qid])
    gid += 1
    sample_id += len(group)
dev_group=dev_extracted['group']

In [14]:
def recall_at_200(preds, dataset):
    global dev_group_rel_num
    global dev_group
    labels = dataset.get_label()
    groups = dataset.get_group()
    assert np.equal(groups, dev_group).all()
    idx = 0
    recall = 0
    for g,gnum in zip(groups, dev_group_rel_num):
        top_preds = labels[idx:idx+g][np.argsort(preds[idx:idx+g])]
        recall += np.sum(top_preds[-200:])/gnum
        idx += g
    assert idx == len(preds)
    return 'recall@200', recall/len(groups), True

In [15]:
dev_extracted['data']['score'] = 0.
for seed in [12345,25523,23543,12352,64654,23123]:
    params = {
            'boosting_type': 'gbdt',
            'objective': 'lambdarank',
            'max_bin':255,
            'num_leaves':63,
            'max_depth':-1,
            'min_data_in_leaf':30,
            'min_sum_hessian_in_leaf':0,
            'bagging_fraction':0.8,
            'bagging_freq':50,
            'feature_fraction':1,
            'learning_rate':0.1,
            'num_boost_round':1000,
            'early_stopping_round':200,
            'metric':'custom',
            'label_gain':[0,1],
            'lambdarank_truncation_level':20,
            'seed':seed,
            'num_threads':max(multiprocessing.cpu_count()//2,1)
    }
    num_boost_round = params.pop('num_boost_round')
    early_stopping_round = params.pop('early_stopping_round')
    gbm = lgb.train(params, lgb_train,
                    valid_sets=lgb_valid,
                    num_boost_round=num_boost_round,
                    early_stopping_rounds=early_stopping_round,
                    feval=recall_at_200,
                    feature_name=feature_name,
                    verbose_eval=True)
    dev_extracted['data']['score'] += gbm.predict(dev_X)
    best_score = gbm.best_score['valid_0']['recall@200']
    print(best_score)
    best_iteration = gbm.best_iteration
    print(best_iteration)
    feature_importances = sorted(list(zip(feature_name,gbm.feature_importance().tolist())),
                                 key=lambda x:x[1],reverse=True)
    print(feature_importances)
eval_recall(dev_qrel, dev_extracted['data'])
eval_mrr(dev_extracted['data'])

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 2.829568 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 11913
[LightGBM] [Info] Number of data points in the train set: 39738110, number of used features: 66
[1]	valid_0's recall@200: 0.695929
Training until validation scores don't improve for 200 rounds
[2]	valid_0's recall@200: 0.72457
[3]	valid_0's recall@200: 0.732295
[4]	valid_0's recall@200: 0.735136
[5]	valid_0's recall@200: 0.738646
[6]	valid_0's recall@200: 0.73984
[7]	valid_0's recall@200: 0.741607
[8]	valid_0's recall@200: 0.742586
[9]	valid_0's recall@200: 0.743541
[10]	valid_0's recall@200: 0.74341
[11]	valid_0's recall@200: 0.744866
[12]	valid_0's recall@200: 0.74489
[13]	valid_0's recall@200: 0.745344
[14]	valid_0's recall@200: 0.745057
[15]	valid_0's recall@200: 0.745714
[16]	valid_0's recall@200: 0.746621
[17]	valid_0's 

[215]	valid_0's recall@200: 0.765521
[216]	valid_0's recall@200: 0.765234
[217]	valid_0's recall@200: 0.765664
[218]	valid_0's recall@200: 0.765592
[219]	valid_0's recall@200: 0.765592
[220]	valid_0's recall@200: 0.765807
[221]	valid_0's recall@200: 0.765521
[222]	valid_0's recall@200: 0.765306
[223]	valid_0's recall@200: 0.765592
[224]	valid_0's recall@200: 0.765449
[225]	valid_0's recall@200: 0.765449
[226]	valid_0's recall@200: 0.765234
[227]	valid_0's recall@200: 0.765449
[228]	valid_0's recall@200: 0.765449
[229]	valid_0's recall@200: 0.765306
[230]	valid_0's recall@200: 0.765162
[231]	valid_0's recall@200: 0.765234
[232]	valid_0's recall@200: 0.764947
[233]	valid_0's recall@200: 0.764947
[234]	valid_0's recall@200: 0.764947
[235]	valid_0's recall@200: 0.764709
[236]	valid_0's recall@200: 0.76478
[237]	valid_0's recall@200: 0.76478
[238]	valid_0's recall@200: 0.764637
[239]	valid_0's recall@200: 0.76478
[240]	valid_0's recall@200: 0.764924
[241]	valid_0's recall@200: 0.76478
[242]

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 3.296402 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 11913
[LightGBM] [Info] Number of data points in the train set: 39738110, number of used features: 66
[1]	valid_0's recall@200: 0.699821
Training until validation scores don't improve for 200 rounds
[2]	valid_0's recall@200: 0.72882
[3]	valid_0's recall@200: 0.733871
[4]	valid_0's recall@200: 0.740962
[5]	valid_0's recall@200: 0.742323
[6]	valid_0's recall@200: 0.744448
[7]	valid_0's recall@200: 0.743804
[8]	valid_0's recall@200: 0.744974
[9]	valid_0's recall@200: 0.745833
[10]	valid_0's recall@200: 0.745905
[11]	valid_0's recall@200: 0.745762
[12]	valid_0's recall@200: 0.747051
[13]	valid_0's recall@200: 0.746693
[14]	valid_0's recall@200: 0.746526
[15]	valid_0's recall@200: 0.746311
[16]	valid_0's recall@200: 0.746693
[17]	valid_0

[215]	valid_0's recall@200: 0.76527
[216]	valid_0's recall@200: 0.765127
[217]	valid_0's recall@200: 0.765055
[218]	valid_0's recall@200: 0.765127
[219]	valid_0's recall@200: 0.76527
[220]	valid_0's recall@200: 0.76527
[221]	valid_0's recall@200: 0.76527
[222]	valid_0's recall@200: 0.765413
[223]	valid_0's recall@200: 0.76527
[224]	valid_0's recall@200: 0.765413
[225]	valid_0's recall@200: 0.765413
[226]	valid_0's recall@200: 0.76527
[227]	valid_0's recall@200: 0.765413
[228]	valid_0's recall@200: 0.765413
[229]	valid_0's recall@200: 0.765556
[230]	valid_0's recall@200: 0.765556
[231]	valid_0's recall@200: 0.7657
[232]	valid_0's recall@200: 0.7657
[233]	valid_0's recall@200: 0.765843
[234]	valid_0's recall@200: 0.765843
[235]	valid_0's recall@200: 0.765843
[236]	valid_0's recall@200: 0.766129
[237]	valid_0's recall@200: 0.7657
[238]	valid_0's recall@200: 0.765413
[239]	valid_0's recall@200: 0.76527
[240]	valid_0's recall@200: 0.76527
[241]	valid_0's recall@200: 0.765413
[242]	valid_0's

[438]	valid_0's recall@200: 0.766297
[439]	valid_0's recall@200: 0.766225
[440]	valid_0's recall@200: 0.766225
[441]	valid_0's recall@200: 0.766225
[442]	valid_0's recall@200: 0.766225
[443]	valid_0's recall@200: 0.766368
[444]	valid_0's recall@200: 0.766368
[445]	valid_0's recall@200: 0.766225
[446]	valid_0's recall@200: 0.766225
[447]	valid_0's recall@200: 0.766225
[448]	valid_0's recall@200: 0.766082
[449]	valid_0's recall@200: 0.766368
[450]	valid_0's recall@200: 0.766368
[451]	valid_0's recall@200: 0.766655
[452]	valid_0's recall@200: 0.766655
[453]	valid_0's recall@200: 0.766511
[454]	valid_0's recall@200: 0.766511
[455]	valid_0's recall@200: 0.766511
[456]	valid_0's recall@200: 0.766798
[457]	valid_0's recall@200: 0.766798
[458]	valid_0's recall@200: 0.766941
[459]	valid_0's recall@200: 0.766798
[460]	valid_0's recall@200: 0.766941
[461]	valid_0's recall@200: 0.766941
[462]	valid_0's recall@200: 0.766941
[463]	valid_0's recall@200: 0.766941
[464]	valid_0's recall@200: 0.766941
[

...

0.7653653295128939
237
[('contents_DFR_In_expB2', 937), ('contents_SCQ_avg', 742), ('contents_Prob', 698), ('contents_DFR_GL2', 525), ('contents_TFIDF_var', 454), ('contents_QueryCoverageRatio', 451), ('contents_SCQ_var', 451), ('contents_LMJM_lambda_0.70', 447), ('contents_NormalizedTF_var', 427), ('contents_NormalizedTF_min', 387), ('contents_LMJM_lambda_0.10', 367), ('contents_LMD_mu_1000', 357), ('contents_TFIDF_max', 345), ('contents_TPscore', 340), ('contents_NormalizedTF_sum', 323), ('contents_UnorderedQueryPairs_15', 320), ('contents_IDF_sum', 305), ('contents_MatchingTermCount', 295), ('contents_BM25_k1_0.90_b_0.40', 293), ('contents_TF_var', 272), ('contents_BM25_k1_2.00_b_0.75', 271), ('contents_UnorderedSequentialPairs_15', 269), ('contents_NTFIDF', 257), ('contents_OrderedSequentialPairs_3', 252), ('contents_OrderedQueryPairs_3', 251), ('contents_DocSize', 248), ('contents_LMJM_lambda_0.40', 244), ('contents_SCQ_sum', 238), ('contents_SCQ_max', 237), ('contents_NormalizedT

100%|██████████| 6980/6980 [01:33<00:00, 74.90it/s]


score_tie occurs 1671795 times in 6978 queries
recall@10:0.4269340974212034
recall@20:0.5224212034383954
recall@50:0.6270176695319962
recall@100:0.7044412607449857
recall@200:0.7664637058261701
recall@500:0.8295606494746895
recall@1000:0.8573424068767909


100%|██████████| 6980/6980 [00:40<00:00, 173.69it/s]

score_tie occurs 862 times in 641 queries 0.2110044571792423
